In [15]:
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, ReLU, Conv2D, Conv2DTranspose, Activation
from tensorflow.keras import Input, Model
from tensorflow.keras.regularizers import l1

In [11]:
def encoder(inputs, layers):
    """ Construct the Encoder
    x: input to the encoder
    layers: number of filters per layer
    """
    outputs = inputs  
    last_filters = layers.pop() # set a side the last layer
    
    # Feature Pooling
    for n_filters in layers:
        outputs = Conv2D(n_filters, (3,3), strides=(2,2), padding='same')(outputs)
        outputs = BatchNormalization()(outputs)
        outputs = ReLU()(outputs)
    
    # Adds sparsity constraints to last layer in encoder
    outputs = Conv2D(last_filters, (3,3), strides=(2,2), padding='same', activity_regularizer = l1(1e-4))(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = ReLU()(outputs)
        
    return outputs

In [14]:
def decoder(inputs, layers):
    """ Construct the Decoder
    inputs: input to the decoder
    layers: the number of filters per layer (in encoder)
    """
    outputs = inputs
    
    #Progressive feature unpooling (dimensionality expansion)
    for _ in range(len(layers)-1, 0, -1):
        n_filters = layers[_]
        outputs = Conv2DTranspose(n_filters, (3,3), strides=(2,2), padding='same')(outputs)
        outputs = BatchNormalization()(outputs)
        outputs = ReLU()(outputs)
        
    # img_out_layers = 3 (colored image) and qual 1 if (gray scale)
    # Last unpooling and restore to image input shape
    outputs = Conv2DTranspose(img_out_layers, (3,3), strides=(2,2), padding='same')(outputs)
    outputs = BatchNormalization()(outputs) 
    outputs = Activation('sigmoid')(outputs)
    return outputs #The decoded image

In [ ]:
layers = [64, 32, 16]
inputs = Input(shape=(rows, columns, channels))
encoding = encoder(inputs, layers)
outputs = decoder(encoding, layers)
model = Model(inputs, outputs)

In [ ]:
# Adds the noise to a copy of the image training data
x_train_noisy = x_train + noise

# Trains the encoder by feeding the noisy images as the training data and the original images as the labels
model.fit(x_train_noisy, x_train, epochs=epochs, batch_size=batch_size, verbose=1)